#### **Взять интеграл с заданной точностью eps:**

In [138]:
import numpy as np
import pandas as pd
from random import random

Запишем условие:

In [139]:
def f(x):
    return np.cos(x)/(x + 2)

Метод трапеций:

In [140]:
def trapezoid(n, f=f, a=0.4, b=1.2):
    h = (b - a) / n
    return sum([h * (f(i) + f(i + h)) / 2 for i in np.arange(a, b, h)])

Используя метод Симпсона, заменим подынтегральную функцию параболой:

In [141]:
def simpsons_method(N, a=0.4, b=1.2):
    h = (b - a) / N
    return sum([h / 6 * (f(i - h / 2) + 4 * f(i) + f(i + h / 2)) for i in np.arange(a + h / 2, b - h / 2 + 1e-5, h)])


Создадим таблицу для проверки сходимости:

In [142]:
data = pd.DataFrame(np.array([[abs(trapezoid(i) - trapezoid(i * 2)) for i in [2 ** j for j in range(1, 7)]],
                            [np.log2(abs(trapezoid(i * 2) - trapezoid(i * 4)) / abs(trapezoid(i) - trapezoid(i * 2))) for i in [2 ** j for j in range(1, 7)]], 
                            [abs(simpsons_method(i) - simpsons_method(i * 2)) for i in [2 ** j for j in range(1, 7)]], 
                            [np.log2(abs(simpsons_method(i * 2) - simpsons_method(i * 4)) / abs(simpsons_method(i) - simpsons_method(i * 2))) for i in [2 ** j for j in range(1, 7)]]]).T, 
                    columns=['eps_tr', 'P_tr', 'eps_simp', 'P_simp'], 
                    index=[2 ** i for i in range(1, 7)])
data

,eps_tr,P_tr,eps_simp,P_simp
2,4.152252e-05,-1.916013,8.296630e-07,-3.991385
4,1.100288e-05,-1.979625,5.216450e-08,-3.997766
8,2.789843e-06,-1.994943,3.265334e-09,-3.999437
16,6.999097e-07,-1.998738,2.041630e-10,-3.999850
32,1.751305e-07,-1.999685,1.276151e-11,-3.999115
64,4.379221e-08,-1.999921,7.980838e-13,-4.000903


ЧТО БУДЕТ, ЕСЛИ В ДАННЫЕ ВНЕСТИ ШУМ?

In [143]:
def trapezoid_noize(n, f=f, a=0.4, b=1.2):
    h = (b - a) / n
    return sum([h * (f(i) + f(i + h)) / 2 + random() for i in np.arange(a, b, h)])

def f_i_noize(i, N, f=f, a=0.4, b=1.2):
    return f(a + ((b - a) * i)/(2 * N))

def simpsons_method_noize(N, a=0.4, b=1.2, f_i=f_i_noize):
    return ((b - a)/(6 * N)) * (f_i(0, N) + 4 * sum([f_i(i, N) + random() for i in range(1, 2 * N + 1)]) + 2 * sum([f_i(i, N) + random() for i in range(2, 2 * N + 2, 2)]) + f_i(2 * N, N))

Посмотрим на сходимость теперь:

In [144]:
data = pd.DataFrame(np.array([[abs(trapezoid_noize(i) - trapezoid_noize(i * 2)) for i in [2 ** j for j in range(1, 7)]],
                            [np.log2(abs(trapezoid_noize(i * 2) - trapezoid_noize(i * 4)) / abs(trapezoid_noize(i) - trapezoid_noize(i * 2))) for i in [2 ** j for j in range(1, 7)]], 
                            [abs(simpsons_method_noize(i) - simpsons_method_noize(i * 2)) for i in [2 ** j for j in range(1, 7)]], 
                            [np.log2(abs(simpsons_method_noize(i * 2) - simpsons_method_noize(i * 4)) / abs(simpsons_method_noize(i) - simpsons_method_noize(i * 2))) for i in [2 ** j for j in range(1, 7)]]]).T, 
                    columns=['eps_tr', 'P_tr', 'eps_simp', 'P_simp'], 
                    index=[2 ** i for i in range(1, 7)])
data

,eps_tr,P_tr,eps_simp,P_simp
2,0.688019,1.270801,0.248547,-1.065511
4,0.845011,0.823436,0.040367,-0.189967
8,4.517143,2.804418,0.027146,-3.400108
16,6.528353,1.013322,0.046431,0.838998
32,13.781177,0.740351,0.021041,-0.596632
64,33.793368,1.224944,0.004663,-0.659913


#### **Вычислить производную:**

In [145]:
def f(x):
    return np.exp(x)

Найдём с помощью правосторонней и двусторонней разностей:

In [146]:
def right(N, x=1):
    h = 10 ** -N
    return (f(x + h) - f(x)) / h

def bilateral(N, x=1):
    h = 10 ** -N
    return (f(x + h) - f(x - h)) / (2 * h)

Сравним результаты работы методов с оригинальным числом e, который является значением производной при x = 1:

In [147]:
print(right(5))
print(bilateral(5))
print(np.e)

2.7182954199567173
2.718281828517632
2.718281828459045


Сразу видим, что двусторонний метод работает лучше.

Создадим таблицу для проверки сходимости:

In [152]:
data = pd.DataFrame(np.array([[abs(right(i) - right(i + 1)) for i in [j for j in range(1, 4)]],
                            [np.log2(abs(right(i + 1) - right(i + 2)) / abs(right(i) - right(i + 1))) for i in [j for j in range(1, 4)]], 
                            [abs(bilateral(i) - bilateral(i + 1)) for i in [j for j in range(1, 4)]], 
                            [np.log2(abs(bilateral(i + 1) - bilateral(i + 2)) / abs(bilateral(i) - bilateral(i + 1))) for i in [j for j in range(1, 4)]]]).T, 
                    columns=['eps_r', 'P_r', 'eps_bi', 'P_bi'], 
                    index=[i for i in range(1, 4)])
data

,eps_r,P_r,eps_bi,P_bi
1,0.126923,-3.369900,4.487431e-03,-6.644577
2,0.012277,-3.326693,4.485188e-05,-6.643865
3,0.001224,-3.322405,4.485161e-07,-6.648103


Как можно увидеть, при N->+inf и, соответственно, при h->0, погрешность eps->0.

ЧТО БУДЕТ, ЕСЛИ В ДАННЫЕ ВНЕСТИ ШУМ?

In [149]:
def right_noize(N, x=1):
    h = 10 ** -N
    return (f(x + random()) - f(x - h + random())) / h

def bilateral_noize(N, x=1):
    h = 10 ** -N
    return (f(x + h + random()) - f(x - h + random())) / (2 * h)

Посмотрим на сходимость теперь:

In [150]:
data = pd.DataFrame(np.array([[abs(right_noize(i) - right_noize(i + 1)) for i in [2 ** j for j in range(1, 5)]],
                            [np.log2(abs(right_noize(i + 1) - right_noize(i + 2)) / abs(right_noize(i) - right_noize(i + 1))) for i in [2 ** j for j in range(1, 5)]], 
                            [abs(bilateral_noize(i) - bilateral_noize(i + 1)) for i in [2 ** j for j in range(1, 5)]], 
                            [np.log2(abs(bilateral_noize(i + 1) - bilateral_noize(i + 2)) / abs(bilateral_noize(i) - bilateral_noize(i + 1))) for i in [j for j in range(1, 5)]]]).T, 
                    columns=['eps_r', 'P_r', 'eps_bi', 'P_bi'], 
                    index=[i for i in range(1, 5)])
data

,eps_r,P_r,eps_bi,P_bi
1,251.836835,3.221902,77.606497,3.708962
2,2214.138754,2.701299,1015.263585,2.979945
3,19786.732595,0.284169,5805.866766,6.010632
4,442145.776241,5.416849,22598.197413,1.103789


Значения расходятся.